# Integrating datasets in Azure SQL Server

### Name: Sasikumar Jayapal
### Datasets: Integrating all datasets(Dataset 1, Dataset 3)
### Student ID: x21153272

In [ ]:
import pandas as pd
import numpy as np

In [2]:
# #defining connection parameters
# import pyodbc 
# server = 'msc-dat-analytics.database.windows.net' 
# database = 'MscDAT' 
# username = 'dap' 
# password = 'India-data-analytics' 
# conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [3]:
#defining connection parameters
import pyodbc 

conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-FMSOI4VL;"
                      "Database=MscDAP;"
                      "Trusted_Connection=yes;")


In [4]:
cursor = conn.cursor()

In [5]:
Master_Disaster_sql = "SELECT DisasterYear,Country_code,Country_Name,Latitude,Longitude,DisasterType,TotalDeaths FROM Natural_Disaster_Events UNION ALL SELECT Year,Country_Code,Country_Name,Latitude,Longitude,Type,Death_Toll FROM Death_Toll_by_Disaster"
df_Master_Disaster_sql = pd.read_sql(Master_Disaster_sql, conn)

In [6]:
cursor.execute('''
IF OBJECT_ID('dbo.Master_Disaster_Event', 'U') IS NOT NULL 
  DROP TABLE dbo.Master_Disaster_Event; 
''')

In [7]:
cursor.execute('''
CREATE TABLE dbo.Master_Disaster_Event(
[Event_ID] [int] IDENTITY(1000,1) primary key,
[Year] [int] NULL,
[Country_Code] [char](3) NULL,
[Country_Name] [varchar](100) NULL,
[Latitude] [float] NULL,
[Longitude] [float] NULL,
[Type] [varchar](50) NULL,
[Death_Toll] [int] NULL,
[Severity] [varchar](30) NULL)
''')

In [8]:
#Inserting dataframe data into dbo.Master_Disaster_Event table
for index, row in df_Master_Disaster_sql.iterrows():
    cursor.execute("INSERT INTO dbo.Master_Disaster_Event(Year,Country_Code,Country_Name,Latitude,Longitude,Type,Death_Toll) values(?,?,?,?,?,?,?)",row.DisasterYear,row.Country_code, row.Country_Name,row.Latitude,row.Longitude,row.DisasterType,row.TotalDeaths)
    conn.commit()

In [9]:
cursor.execute('''
MERGE dbo.Master_Disaster_Event AS TARGET
USING dbo.RISK_COUNTRY AS SOURCE
ON(TARGET.Country_code = SOURCE.Country_Code)
WHEN MATCHED 
THEN UPDATE SET 
TARGET.Severity = SOURCE.Risk_Category;
''')
conn.commit()

In [10]:
Master_Disaster_Event_sql = "SELECT * FROM dbo.Master_Disaster_Event"
df_Master_Disaster_Event_sql = pd.read_sql(Master_Disaster_Event_sql, conn)

In [11]:
cursor.close()

In [12]:
df_Master_Disaster_Event_sql.to_csv("Master_Disaster_Events.csv")